In [ ]:
# import subprocess, sys
# subprocess.check_call([sys.executable, "-m", "pip", "install", "pandas", "openpyxl", "-q"])

In [ ]:
import pandas as pd
import glob

# load scores first so we can match picks files to actual weeks
scores = pd.read_csv("spreadspoke_scores.csv")
scores = scores[
    (scores["schedule_season"] == 2025)
    & (scores["schedule_week"].str.isnumeric())
].reset_index(drop=True)
scores = scores[scores["schedule_week"].astype(int) <= 18].reset_index(drop=True)
print(f"{len(scores)} regular-season games in 2025 season")

# map team names to IDs
teams = pd.read_csv("nfl_teams (1).csv")
name_to_id = teams.set_index("team_name")["team_id"].to_dict()
scores["team_home"] = scores["team_home"].map(name_to_id).replace("LVR", "LV")
scores["team_away"] = scores["team_away"].map(name_to_id).replace("LVR", "LV")
scores["team_favorite_id"] = scores["team_favorite_id"].replace("LVR", "LV")
scores["game"] = scores["team_away"] + " @ " + scores["team_home"]
scores["week"] = scores["schedule_week"].astype(int)

# build week -> set of games lookup
week_games = {w: set(grp["game"]) for w, grp in scores.groupby("week")}

# load picks, matching each file to its actual NFL week
files = glob.glob("picks_export*.xlsx")
print(f"Found {len(files)} picks files")

all_rows = []
for f in files:
    raw = pd.read_excel(f)
    file_games = set(raw.columns[1:])
    # find the week with most game overlap
    week = max(week_games, key=lambda w: len(file_games & week_games[w]))

    spreads = raw.iloc[0, 1:]
    games = raw.columns[1:]
    for _, row in raw.iloc[1:].iterrows():
        player = row["Super Bowl"]
        for game in games:
            pick_raw = row[game]
            if pd.isna(pick_raw):
                continue
            parts = str(pick_raw).strip().rsplit(" ", 1)
            team = parts[0]
            confidence = int(parts[1].strip("()")) if len(parts) > 1 else None
            all_rows.append({
                "week": week,
                "player": player,
                "game": game,
                "spread": spreads[game],
                "pick": team,
                "confidence": confidence,
            })

picks = pd.DataFrame(all_rows)
picks["spread"] = pd.to_numeric(picks["spread"], errors="coerce")
print(f"{len(picks)} pick rows, {picks['week'].nunique()} weeks, {picks['player'].nunique()} players")
picks.head()

In [ ]:
# (scores and teams already loaded in cell above)

In [ ]:
# (team ID mapping already applied in cell above)

In [ ]:
# get the platform spread per game from picks (one spread per week+game)
platform_spread = picks.groupby(["week", "game"])["spread"].first().reset_index()
platform_spread.columns = ["week", "game", "platform_spread"]

# merge platform spread onto scores
scores = scores.merge(platform_spread, on=["week", "game"], how="left")

# ATS winner using platform spread
# spread convention: negative = home favored, positive = away favored
# margin + spread > 0 → home covers, < 0 → away covers, == 0 → push
def ats_winner(r):
    spread = r["platform_spread"]
    if pd.isna(spread) or pd.isna(r["score_home"]) or pd.isna(r["score_away"]):
        return None
    margin = r["score_home"] - r["score_away"]
    result = margin + spread
    if result > 0:
        return r["team_home"]
    elif result < 0:
        return r["team_away"]
    else:
        return "PUSH"

scores["ats_winner"] = scores.apply(ats_winner, axis=1)

# check if each player's pick matches the ATS winner
picks_with_result = picks.merge(
    scores[["week", "game", "ats_winner"]], on=["week", "game"], how="inner"
)
picks_with_result["correct"] = (picks_with_result["pick"] == picks_with_result["ats_winner"]).astype(int)

# pivot to wide: one column per player
player_cols = picks_with_result.pivot_table(
    index=["week", "game"], columns="player", values="correct", aggfunc="first"
)

# merge player columns onto the full scores table
scores_and_picks = scores.merge(player_cols, on=["week", "game"], how="left")
scores_and_picks = scores_and_picks.sort_values(["week", "game"]).reset_index(drop=True)

players = [c for c in player_cols.columns]
print(f"{len(scores_and_picks)} games, {len(players)} players as columns")
scores_and_picks.head(10)

In [ ]:
scores_and_picks.to_csv("scores_and_picks.csv", index=False)
picks.to_csv("picks.csv", index=False)
print(f"Wrote scores_and_picks.csv ({len(scores_and_picks)} rows) and picks.csv ({len(picks)} rows)")